In [1]:
import pdb
import os
import sys
from ctypes import (
    CDLL,
    POINTER,
    Structure,
    byref,
    string_at,
    c_char_p,
    c_int32,
    c_int64,
    c_uint64,
    c_ubyte,
    cast
)
import json

from ctypes.util import find_library
from typing import Optional, Union

LIB: CDLL = None

class FfiByteBuffer(Structure):
    """A byte buffer allocated by python."""
    _fields_ = [
        ("length", c_int64),
        ("data", POINTER(c_ubyte)),
    ]


class FfiError(Structure):
    """An error allocated by python."""
    _fields_ = [
        ("code", c_int32),
        ("message", c_char_p),
    ]


def _decode_bytes(arg: Optional[Union[str, bytes, FfiByteBuffer]]) -> bytes:
    if isinstance(arg, FfiByteBuffer):
        return string_at(arg.data, arg.length)
    if isinstance(arg, memoryview):
        return string_at(arg.obj, arg.nbytes)
    if isinstance(arg, bytearray):
        return arg
    if arg is not None:
        if isinstance(arg, str):
            return arg.encode("utf-8")
    return bytearray()


def _encode_bytes(arg: Optional[Union[str, bytes, FfiByteBuffer]]) -> FfiByteBuffer:
    if isinstance(arg, FfiByteBuffer):
        return arg
    buf = FfiByteBuffer()
    if isinstance(arg, memoryview):
        buf.length = arg.nbytes
        if arg.contiguous and not arg.readonly:
            buf.data = (c_ubyte * buf.length).from_buffer(arg.obj)
        else:
            buf.data = (c_ubyte * buf.length).from_buffer_copy(arg.obj)
    elif isinstance(arg, bytearray):
        buf.length = len(arg)
        if buf.length > 0:
            buf.data = (c_ubyte * buf.length).from_buffer(arg)
    elif arg is not None:
        if isinstance(arg, str):
            arg = arg.encode("utf-8")
        buf.length = len(arg)
        if buf.length > 0:
            buf.data = (c_ubyte * buf.length).from_buffer_copy(arg)
    return buf


def _load_library(lib_name: str) -> CDLL:
    lib_prefix_mapping = {"win32": ""}
    lib_suffix_mapping = {"darwin": ".dylib", "win32": ".dll"}
    try:
        os_name = sys.platform
        lib_prefix = lib_prefix_mapping.get(os_name, "lib")
        lib_suffix = lib_suffix_mapping.get(os_name, ".so")
        lib_path = os.path.join(
            os.path.dirname(os.getcwd()), f"agora-allosaurus-rs/target/release/{lib_prefix}{lib_name}{lib_suffix}"
        )
        return CDLL(lib_path)
    except KeyError:
        print ("Unknown platform for shared library")
    except OSError:
        print ("Library not loaded from python package")

    lib_path = find_library(lib_name)
    if not lib_path:
        if sys.platform == "darwin":
            ld = os.getenv("DYLD_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.dylib")
            if os.path.exists(lib_path):
                return CDLL(lib_path)

            ld = os.getenv("DYLD_FALLBACK_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.dylib")
            if os.path.exists(lib_path):
                return CDLL(lib_path)
        elif sys.platform != "win32":
            ld = os.getenv("LD_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.so")
            if os.path.exists(lib_path):
                return CDLL(lib_path)

        raise Exception(f"Error loading library: {lib_name}")
    try:
        return CDLL(lib_path)
    except OSError as e:
        raise Exception(f"Error loading library: {lib_name}")


def _get_library() -> CDLL:
    global LIB
    if LIB is None:
        LIB = _load_library("agora_allosaurus_rs")

    return LIB

def _get_func(fn_name: str):
    return getattr(_get_library(), fn_name)

def _free_buffer(buffer: FfiByteBuffer):
    lib_fn = _get_func("allosaurus_byte_buffer_free")
    lib_fn(byref(buffer))


def _free_string(err: FfiError):
    lib_fn = _get_func("allosaurus_string_free")
    lib_fn(byref(err))


def _free_handle(handle: c_int64, err: FfiError):
    lib_fn = _get_func("allosaurus_create_proof_free")
    lib_fn(handle, byref(err))

In [2]:
def new_server() -> c_int64:
    err = FfiError()
    lib_fn = _get_func("allosaurus_new_server")
    lib_fn.restype = c_uint64

    handle = lib_fn(byref(err))
    if handle == 0:
        message = string_at(err.message)
        raise Exception(message)
    handle = c_uint64(handle)
    return handle
server = new_server()
server

c_ulong(4707129610607788034)

In [3]:
def new_user(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_new_user")
    lib_fn(server, byref(buffer), byref(err))

    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
user = new_user(server)
user

b'\x1c\xb5oo\xf8\xd3"k\xaf\x0b\xb6\xe9Lc\x8f\xe5sOLX;]\x8f\x83+\xbc\xb2W\xb0!\x90-\x00\x98c\xb5\x87\x00\x8e\x07\x04)\xa0z\xfd^\x06\xc39\xa7\x01\xc3P6\x8ds3\x02\xf4wMl,x\xdck\xc7f0Ha\xbe\xe9\xf5?G\xf6\xa9\x90\xcfK\x98\xc1\x96_Rq\x1bj\x1f\x0b\xee!OS\xe4\xee\xfaH0\xb9\xc2\xb0^\x90*\x9f\x08\x8b\xb5/\xb2\xcfU0\xa2K\xb9+q\xcd\x1e\xd8c<M\x94\x84<\x07\x9d\xfc\xa4P\x85\xa0\x0c\xf2\x9a\xd05\x80\xf4xh\xbd\xc8\x1a\xb2\xc6rmoN\xecNs\xf1\x15\xb8/\xaaj8\x8f\x82\x08\xcd\x92\'\x9b\x0ej\r\xc7\xa8\xeb\x95\xc0\xc1\xd7\x98\x88y\xdd\x18\xda\xe8Cl\x17\xc7u]\xcc7+\x01W\x9b\xd2\x1a ;\x7f\xc9\xdd|\xb6e\x1a7\xe8\x03\xfa1t\xd2{\xfcV\x9fgE\xd5\x0e\xa7f7\xb6v\xae\xe7\xf3\'\xaa\xc4\x81o?Sfb\xe0S\x91\x9c7\xbaf\xfeO\x12\xfbfh\x1f6@]`\xc2\x90\x0e\xfb\xbe\xeci\x8bt\xbb \x19\x01'

In [4]:
def server_add(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_add")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_add(server, user)

b'\xac\xf5\xf4\xe6\xfe\x15\xa6\xdf\xd3qX\x08\xb2\xa2,\x84\xe6\x03\x9aS{\x1aT\xacE=!\x9as\xb0(&\xba\xabi\xa1\xf9RR\xd0f\xd9\xdd\xba\xee\xaa\xa2)'

In [5]:
def server_delete(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_delete")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
# server_delete(server, user)

In [6]:
def user_create_witness(server, user):
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_user_create_witness")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
user = user_create_witness(server, user)

In [7]:
def check_witness(user):
    lib_fn = _get_func("allosaurus_user_check_witness")
    err = lib_fn(_encode_bytes(user))
    if err != 0:
        raise Exception("Witness is invalid")
    return "Witness is valid"
check_witness(user)

'Witness is valid'

In [8]:
def user_make_membership_proof(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    challenge = bytearray(os.urandom(32))
    lib_fn = _get_func("allosaurus_user_make_membership_proof")
    lib_fn(server, _encode_bytes(user), _encode_bytes(challenge), byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
proof = user_make_membership_proof(server, user)
proof

b'\x98E\xf6\x11.\x86v\xf6N<\x7f\x83\x82\xc9\xedIX\xc9\xe9Y\x0f\x9d\x13\x97\xd7\xd7\xc2\x9c6\x11.\xcd\x91\xfe\xcb\x972M\x97h\x06`E,$./\x8d\x98E\xb5H\xe0\xaf-e\xd7\xde\x11\xef6\x00Q\xd6\xb7\xb6A\x98\x80io\x0c\xa4\xb7\xfa\x1e+\xd0\x0ccV<sI\xa3\xb7F\xc93J\x8d8|\x9d\xc3M\xb0\x86\x15-\xec\xa0\x04\xb5w\xdcT\x146\xa1\xb7&k\xd7\x07;\xd5\xc1\x19T\xbb\x03\x1f\x90\x8b\xd1\x88\x1c\xf2S\x9aF\x1b\xe0Q}*h\x96\x0f\x18\x8e9\xe5$L\x7fi\xf1/\xef_&\xd8yK\xec\xda:\x16\n@\xec\x9b\xeb\x97^\xe5\xfd\x8c\xf5Z\xd7\x7f\x9c)\x00\x047\xf1\x16\xe5\x81\xc3qw"Q\xb7W\xbcHc\x87\xcb\xcb5^6s\t\x13\xd0m\'.s\x94\x1e\xf0\xddevEp\x810}\xd0\xdfy\xa7\x02\x8dW\x93\xf0\x970\x19,\x9cK\xf1j\xa5\x99\x0c\xc6\xd9\x12\x18\xd0\xed\xde=\xf3$6\xfd\xdc\xffou\xb3:\x82v\xb2\xbb\xcf\xa6\xc6P\xfe\x96\x90C\x04\x03gS\x05Q\x80\xce\x94y~G[\x97\xb5\xa46A\x91?V\xa1,tT\x92\xdd\x01\xad\xc2MU\n9QK\x1a\x86\x17\xd5\xc7J\x03\xb6\xe4&\xd2\xd6T\xaa\xee\xd4\x04\'\x02q\xdc,C\x82\xb0\xde\xbb^\xc1\xadw\xea5\xa8\xcc\x03\r\xbd\xa8\x97\xa0\xbb\xdf\xcd^\r7\xb1\xe5)H

In [9]:
def witness_check_membership_proof(server, proof) -> c_int64:
    err = FfiError()
    lib_fn = _get_func("allosaurus_witness_check_membership_proof")
    lib_fn(server, _encode_bytes(proof), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    return "Membership proof verified successfully"
witness_check_membership_proof(server, proof)

'Membership proof verified successfully'

In [10]:
def user_update(servers, user, threshold) -> c_int64:
    array_type = c_uint64 * len(servers)
    servers = array_type(*servers)

    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_user_update")
    lib_fn(servers, len(servers), _encode_bytes(user), threshold, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
    

In [11]:
def server_batch_delete(server, user_list):
    user_buffer = (FfiByteBuffer * len(user_list))()
    for i, tmp_user in enumerate(user_list):
        # tmp_user = _encode_bytes(user)
        array_type = c_ubyte * len(tmp_user)
        c_array = array_type(*tmp_user)
        user_buffer[i].length = len(tmp_user)
        user_buffer[i].data = cast(c_array, POINTER(c_ubyte))

    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_update")
    lib_fn(user_buffer, len(user_list), server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer


In [12]:
user1 = new_user(server)
user2 = new_user(server)
user3 = new_user(server)  
server_add(server, user1)


b'\x8f\xd4-/$\xf8]\xdd\x93<\xe4)\xce\x17\x9bD\xc3%\xe3\x18\xd3l\xaf\xa6\xe7\xbajP\x98\xc8O\x1cI\x9c\x06\x16\x91\xda\xc3(\xca?\xde\x05r\x8d\xb2j'

In [13]:
server_batch_delete(server, [user1, user2, user3]) # nedd to be c array

user_ids: [Scalar(0x6083ecd9300dd8bb7ac7c2c4594437d4e317585162e07d203394962cd272858c), Scalar(0x35343a376b53624745d16cca2dee909b8fca0e7aa67b19177cd04b2167d4205c), Scalar(0x3cf8335ac99c8a737aa40ff2d7c660b9d8177efafb8220d619191c2d46babf0f)]
ds: [], vs: []


b'\x00\x00'

In [6]:
def server_get_epoch(server) -> int:
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_epoch")
    return lib_fn(server, byref(err))
server_get_epoch(server)

1

In [7]:
def server_get_accumulator(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_accumulator")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_accumulator(server)

b'\xb0\xe1\x10\xdc98\xb3\x12rF\xc7L\x94\xff\x8e\x96=L\x03\xe6\xda\xdf\xb4\xe13\xe4\x99d\xbd\xcf\xea\x1ax\xf2^\x88l\xf6\xf3\xca6T\x08N\x11GNK'

In [8]:
def server_get_witness_public_key(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_witness_public_key")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_witness_public_key(server)

b"\x93\x9c9F\xee\xd3@\x03\xfb\x8e\xbb\xc1\xfa\xc5^@\xc6\xf0t\x81#Y\x93WC}\xd3\x8a\x91\xdb.\xa3~\xfb\xce\xdf\xd1\x1b\xe6q\x969,&\x8dP\x83\xfe\x0c8{\xbcf\xe5\xf5\x9bbM\xcccc\xccH\xb3\xd9\x02\xaa5\x032'\xfc\xe5\xabT\x14\xc7\xf3\xc8T\x82_\xfe\xedN\xfa\x08\x9d&\x04\xd1=\x03\xb6\xd7A"

In [9]:
def server_get_sign_public_key(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_sign_public_key")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_sign_public_key(server)

b'\xad\x94*To\x87K\x01"\xa3\x01\x9a\xef}\x04\xcf5Z\xc3\xe0yN\xbf\x04K\xc7G\x05\x7f&b\xf1\xb0\xec\xad\xa1\xb9\xbc8b\xdf\xfaP`PFQ\x7f\x16i\xcbi.\xe7\xe9\x97oX\xdf\xc4\xa54\xad\xa6s\x99\x16\x14&#3\xe6\xf7L\x8cA\x99\n5d\x05\xdeg\x7f\xef\xb2\x8a\xf1~\x0cG\x8ca\n\xfd\x16'

In [10]:
def server_get_public_keys(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_public_keys")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_public_keys(server)

b'\x93\x9c9F\xee\xd3@\x03\xfb\x8e\xbb\xc1\xfa\xc5^@\xc6\xf0t\x81#Y\x93WC}\xd3\x8a\x91\xdb.\xa3~\xfb\xce\xdf\xd1\x1b\xe6q\x969,&\x8dP\x83\xfe\x0c8{\xbcf\xe5\xf5\x9bbM\xcccc\xccH\xb3\xd9\x02\xaa5\x032\'\xfc\xe5\xabT\x14\xc7\xf3\xc8T\x82_\xfe\xedN\xfa\x08\x9d&\x04\xd1=\x03\xb6\xd7A\xad\x94*To\x87K\x01"\xa3\x01\x9a\xef}\x04\xcf5Z\xc3\xe0yN\xbf\x04K\xc7G\x05\x7f&b\xf1\xb0\xec\xad\xa1\xb9\xbc8b\xdf\xfaP`PFQ\x7f\x16i\xcbi.\xe7\xe9\x97oX\xdf\xc4\xa54\xad\xa6s\x99\x16\x14&#3\xe6\xf7L\x8cA\x99\n5d\x05\xdeg\x7f\xef\xb2\x8a\xf1~\x0cG\x8ca\n\xfd\x16'